# Pandas Intermediate

In [4]:
import pandas as pd
reviews = pd.read_csv("../input/wine_reviews/winemag-data-130k-v2.csv", index_col=0)

In [ ]:
# Làm cho Notebook rộng hơn một chút, 
# cần thiết nếu dùng màn hình rộng
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

## Các phân tích với nhóm
Khi muốn nhóm dữ liệu lại, chúng ta có thể dùng hàm `groupby()`. Trong slide trước chúng ta đã dùng hàm `value_counts()` để đếm số lần xuất hiện của một giá trị nào đó. Một cách làm khác tương tự là:

In [ ]:
reviews.groupby('points').points.count()
reviews.groupby('country').max()

`groupby()` tạo một nhóm các reviews có cùng điểm đánh giá. Hàm `groupby()` không thực hiện thay đổi gì với bảng mà chỉ trả về một biến dạng `DataFrameGroupBy`, và ta cần nối với một hàm nào đó có sẵn (`sum, max, mean, min`) hoặc tự viết với `apply`. 

Với ví dụ trên, chúng ta lấy cột `points()` và đếm bao nhiêu lần giá trị này xuất hiện. `value_counts()` là hàm cho ra kết quả tương tự với cách `groupby()` này.

Hãy xem một ví dụ nhỏ khác sau đây:

In [5]:
import seaborn as sns
iris_data = sns.load_dataset('iris')
df = iris_data.head(5).copy()
df = pd.concat([df, iris_data.iloc[50:55]])
df = pd.concat([df, iris_data.iloc[100:105]])

In [6]:
df.groupby('species').sum()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,24.3,16.4,7.0,1.0
versicolor,32.3,14.6,22.7,7.2
virginica,32.0,14.9,28.4,10.5


![Ảnh](https://miro.medium.com/max/2000/0*m-puRNwcnUl5zZ4O)

Ta có thể chỉ định cột nào sẽ được tính tổng. So sánh 2 cách sau:

In [ ]:
df.groupby(‘species’).sum()[‘sepal_width’] # ← BAD!
df.groupby(‘species’)[‘sepal_width’].sum() # ← BETTER & FASTER!

Chúng ta có thể dùng bất kỳ hàm tổng hợp nào với dữ liệu này. Vd để liệt kê các loại vang rẻ tiền nhất trong mỗi nhóm điểm, ta làm `reviews.groupby('points').price.min()`

Mỗi một nhóm tạo ra bởi `groupby()` có thể được xem như một slice của DataFrame chứa các giá trị trùng nhau. Và DataFrame này có thể được truy cập trực tiếp dùng hàm `apply()`, và ta có thể thao tác trên dữ liệu này theo cách ta muốn. Vd, một cách để chọn tên của rượu vang đầu tiên được reviewed từ mỗi lò rượu:

In [7]:
reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

winery
1+1=3                                     1+1=3 NV Rosé Sparkling (Cava)
10 Knots                            10 Knots 2010 Viognier (Paso Robles)
100 Percent Wine              100 Percent Wine 2015 Moscato (California)
1000 Stories           1000 Stories 2013 Bourbon Barrel Aged Zinfande...
1070 Green                  1070 Green 2011 Sauvignon Blanc (Rutherford)
                                             ...                        
Órale                       Órale 2011 Cabronita Red (Santa Ynez Valley)
Öko                    Öko 2013 Made With Organically Grown Grapes Ma...
Ökonomierat Rebholz    Ökonomierat Rebholz 2007 Von Rotliegenden Spät...
àMaurice               àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                                    Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

Để truy cập sâu hơn, ta có thể nhóm với hơn một cột. Vd để chọn dòng vang tốt nhất theo nước và tỉnh:

In [ ]:
# Dùng hàm không tên với cú pháp lambda (Unnamed function)
# Ở đây ta hiển thị tất cả các cột còn lại (sau khi group) của DataFrame
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

In [12]:
# Dùng hàm có tên get_best_wine tương đương với cách trên (Named function)
def get_best_wine(group):
    return group.loc[group.points.idxmax()]

# Ở đây, ta chỉ hiển thị 2 cột 
reviews.groupby(['country', 'province'])['points', 'price'].apply(get_best_wine)   

# tại sao câu lệnh dưới báo lỗi?? không work với 1 cột points??
# reviews.groupby(['country', 'province'])['points'].apply(get_best_wine)   

points  price
country   province                       
Argentina Mendoza Province    97.0  120.0
          Other               95.0   90.0
Armenia   Armenia             88.0   15.0
Australia Australia Other     93.0   15.0
          New South Wales     94.0   32.0
...                            ...    ...
Uruguay   Juanico             90.0   45.0
          Montevideo          91.0   60.0
          Progreso            90.0   46.0
          San Jose            87.0   50.0
          Uruguay             91.0   22.0

[425 rows x 2 columns]

Một hàm nữa cần phải trao đổi là `agg()`. Hàm này cho phép chạy nhiều tính toán khác nhau trên DataFrame cùng lúc. Ví dụ có thể tạo một bảng tổng hợp thống kê đơn giản của dataset như sau:

In [ ]:
# reviews.groupby('winery').title.agg(max)
reviews.groupby(['country']).price.agg([len, min, max])

Sử dụng thông thạo `groupby()` sẽ cho phép bạn làm nhiều điều hoành tráng với dataset của mình.

## Multi-indexes
Groupby() với nhiều hơn một cột sẽ cho ta DataFrame hoặc Series với multi-index. 
Multi-index khác với index đơn là nó có nhiều levels.

In [ ]:
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed

In [ ]:
mi = countries_reviewed.index
type(mi)

Cần 2 nhãn để truy xuất một giá trị của đối tượng có multi-index, xem thêm ở [doc của pandas](https://pandas.pydata.org/pandas-docs/stable/advanced.html). **Trong thực tế, hàm được dùng nhiều nhất lại là reset_index() . Why??**

## Sắp xếp
Xem lại `countries_reviewed` ta thấy nhóm trả về dữ liệu theo thứ tự của index, không phải của giá trị. Để có dữ liệu theo thứ tự mong muốn, ta có thể sắp xếp dùng hàm `sort_values()` (mặc định là theo thứ tự tăng dần, tuy nhiên, đa số các trường hợp chúng ta sẽ muốn hiển thị theo thứ tự giảm dần, nên sẽ phải thêm đối số `ascending`)

In [ ]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed
# countries_reviewed.sort_values(by='len', ascending=False)

Để sắp xếp theo giá trị index, ta dùng hàm tương đương `sort_index()`: `countries_reviewed.sort_index()`. Và cũng giống như `groupby()`, ta có thể sort theo nhiều cột.

In [ ]:
countries_reviewed.sort_values(by=['country', 'len'])

## Bài tập thực hành
### 1
Ai là người review vang nhiều nhất? Tạo một `Series` với index là `taster_twitter_handle` và giá trị là số lần mỗi người review viết.
(gợi ý: dùng `groupby` với `size()` hoặc `count()`)

In [ ]:
reviews_written = 

### 2
Giả sử tôi có $400, tìm tên loại vang ngon nhất mà tôi có thể mua với số tiền đang có? Tạo một `Series` có index là giá vang và giá trị là số điểm tối đa mà loại vang có giá chừng đó được đánh giá trong một review. Sắp xếp giá trị theo giá, thứ tự tăng dần)
(gợi ý: dùng `max()` và `sort_index()` với các cột liên quan là `price` và `points`)

In [ ]:
best_rating_per_price = 

### 3
Giá tiền thấp nhất và cao nhất cho mỗi `variety` của vang là gì? Tạo một `DataFrame` có index là danh mục `variety` và giá trị là `min` và `max` của giá. (gợi ý: dùng `agg()`)

In [ ]:
price_extremes = 

### 4
Những loại vang đắt tiền nhất trong mỗi nhóm là gì? Tạo biến `sorted_varieties` với giá trị từ câu trước được sắp xếp theo thứ tự giảm dần dựa vào giá min, sau đó là giá max.

In [ ]:
sorted_varieties =

### 5
Tạo một `Series` có index là những người review và values là điểm review trung bình của người đó (gợi ý: cần các cột `taster_name` và `points`)

In [ ]:
reviewer_mean_ratings =

### 6
Nhóm nước và loại vang nào là phổ biến nhất? Tạo một `Series` có index là một `MultiIndex` của cặp `{country, variery}`. Sắp xếp giá trị torng `Series` theo thứ tự giảm dần dựa vào số lượng vang đếm được. (gợi ý: dùng `groupby()` rồi đếm trên `variety` sau đó `sort_values()`.)

In [ ]:
country_variety_counts =

## Tìm hiểu kiểu dữ liệu
### dtypes

In [ ]:
print(reviews.price.dtype)
reviews.dtypes

Những cột chỉ chứa chuỗi không có kiểu dữ liệu riêng, mà ở đây được gán kiểu `object`. Tương tự như numpy, ta có thể ép kiểu dùng `astype()`

In [ ]:
reviews.points.astype('float64')

### Các hàm hỗ trợ chuyển đổi khác
* `pd.to_numeric()` và
* `pd.to_datetime()`

Ưu điểm chính của chúng so với `astype` là giúp ta chỉ ra hành động cần làm khi một giá trị không thể chuyển đổi được. Cả 2 hàm có đối số `errors` cho phép ta bỏ qua lỗi với `errors='ignore'` hoặc chuyển giá trị đó thành `np.nan` với cách truyền `errors='coerce'`. Mặc định là pandas sẽ báo lỗi.

## Đối mặt với dữ liệu không đầy đủ
Những chỗ dữ liệu bị thiếu, không đầy đủ thì được pandas hiển thị là `NaN` (viết tắt cho `Not a Number`). Kiểu của các giá trị `NaN` này luôn là `float64`.

Để chọn các mục `NaN` này, ta dùng hàm `pd.isnull()`

In [ ]:
reviews[pd.isnull(reviews.country)]

Thay thế các giá trị bị thiếu này là một tác vụ thường xuyên với `fillna()`. Vd ta có thể thay thế mỗi giá trị `NaN` với `Unknown`:

In [ ]:
reviews.region_2.fillna("Unknown")

Chúng ta có thể thay thế giá trị thiếu với giá trị non-null nào đó trong dataset. Đây gọi là lấp ngược (backfill strategy). Thêm vào đó, ta có thể dùng `replace()` để thay thế giá trị hiện tại. Vd: cập nhật tên tài khoản Twitter của người review rượu.

In [ ]:
reviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")

### `pd.to_numeric()`
Chúng ta thử cố tình làm sai lệch dữ liệu đi:

In [ ]:
reviews.loc[34285, 'price'] = 'I am causing trouble'
reviews.loc[13285, 'price'] = 'Me too'

Bây giờ thì `reviews.price.astype(int)` sẽ fail vì pandas không biết chuyển đổi chuỗi sang số integer. Trường hợp này, tôi thường kiểm tra các giá trị trong cột để hiểu rõ hơn chúng ta đang gặp phải những gì

In [ ]:
reviews.price.apply(lambda x: type(x)).value_counts()

Ta có thể xác định các hàng có dữ liệu bị sai bằng cách sau. Và từ đó có thể nhanh chóng sửa chúng cho phù hợp.

In [ ]:
reviews['price'][reviews.price.apply(lambda x: isinstance(x,str))]

In [ ]:
# chuyển đổi thành np.nan rồi điền dữ liệu phù hợp
reviews.price = pd.to_numeric(reviews.price, errors='coerce')
reviews.price = reviews.price.fillna(reviews.price.median())
reviews.price.astype(int)

In [ ]:
reviews['reviewed_date'] = '20170108'

### `pd.to_datetime()`
Hàm này chuyển chuỗi thành dạng ngày tháng. Pandas sẽ đoán định dạng và lấy ngày tháng ra từ chuỗi đầu vào.

In [ ]:
print(pd.to_datetime('2019-8-1'))
print(pd.to_datetime('2019/8/1'))
print(pd.to_datetime('8/1/2019'))
print(pd.to_datetime('Aug, 1 2019'))
print(pd.to_datetime('Aug - 1 2019'))
print(pd.to_datetime('August - 1 2019'))
print(pd.to_datetime('2019, August - 1'))
print(pd.to_datetime('20190108'))

Tuy nhiên, chúng ta sẽ thường gặp các định dạng không giống ai, ví dụ như trong dòng cuối cùng kia, khi đó ta phải dùng `pd.to_datetime('20190108',format='%Y%d%m')` để chuyển đổi. Lưu ý một đối số quan trọng khi làm việc với định dạng ngày tháng `exact=False` sẽ giúp chuyển đổi đúng kể cả khi chuỗi đầu vào là `yolo 20190108`. Chúng ta hãy chuyển đổi cột `reviewed_date` mới thêm vào:

In [ ]:
reviews.reviewed_date = pd.to_datetime(reviews.reviewed_date, format='%Y%d%m')

Khi đó, ta có thể dùng accessor `dt` để truy cập đến muôn kiểu định dạng ngày tháng liên quan như: `date, weekday_name,days_in_month,second,hour` v.v...

In [ ]:
reviews.reviewed_date.dt.month_name()

Tương tự ta có accessor `str` cho chuỗi với rất nhiều hàm `lower()`, `upper()`, `center(width)`, `startswith(<substring>)`, `swapcase()` và `cat` cho categories (ít dùng)

## Bài tập thực hành
1. Tạo một Series từ những giá trị của cột `points` nhưng chuyển các giá trị này sang kiểu strings. (trong Python là `str`)
2. Đôi khi giá rượu bị để là null. Có bao nhiêu reviews trong dataset bị thiếu giá?
3. Những vùng nào sản xuất vang thông dụng nhất? Tạo một Series đếm số lần mỗi giá trị xuất hiện trong cột `region_1`. Các giá trị này thường bị thiếu, nên thay những giá trị đó bằng `Unknown`. Sắp xếp giảm dần. Kết quả sẽ giống thế này
```
Unknown                    21247
Napa Valley                 4480
                           ...  
Bardolino Superiore            1
Primitivo del Tarantinsso        1
Name: region_1, Length: 1230, dtype: int64
```
4. Tìm hiểu lỗi của câu lệnh sau và sửa nó

`reviews.taster_name.apply(lambda x: x.split(' ')[-1])`

5. Đổi tên taster_name từ NaN của tất cả các dòng < 100k thành "Robert Smith" và >= 100k thành "Alan Greenspan"

## Đổi tên
Thường thì dữ liệu ta có sẽ được đặt tên cột, tên hàng theo cách mà ta không quen, không thích. Khi đó, pandas có những hàm giúp ta đổi tên khác hay ho hơn. `rename()` cho phép ta đổi tên cột hay index dùng keyword `index` hay `column`

In [ ]:
reviews = reviews.rename(columns={'points':'score'})
reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})

Ta sẽ thường phải đổi tên cột, nhưng rất hiếm khi đổi tên index. Nên thường hàm `set_index()` hay được dùng vì thuận tiện hơn.

Bản thân tên cột và tên hàng cũng có thuộc tính `name` riêng của chúng. Hàm `rename_axis()` có thể được dùng để đổi các tên này. Vd:

In [ ]:
reviews.rename_axis("STT", axis='rows').rename_axis("Trường", axis='columns')

## Kết hợp dữ liệu
Pandas cung cấp 3 cách chính để phục vụ nhu cầu kết hợp nhiều DataFrames hay Series lại với nhau: `concat()`, `join()`, và `merge()`. Hầu hết những gì `merge()` có thể làm cũng có thể được thực hiện với `join()` nên ta sẽ tập trung vào 2 cách đầu tiên.
### concat()
Gộp dữ liệu theo chiều dọc. Cách này hữu dụng khi ta có dữ liệu nằm trong các đối tượng Series hay DataFrame khác nhau nhưng có cùng số lượng cột, sau khi concat thì số dòng sẽ tăng lên nhưng số cột giữ nguyên.
### join()
Hàm này phức tạp hơn một chút và cho phép ta gộp nhiều đối tượng DataFrame khác nhau nhưng có cùng chung một index. Tức là sẽ tăng số cột theo chiều ngang. Để dùng `join` ta cần khai báo cách mà dữ liệu được gộp lại
* **Left**: Giữ tất cả hàng trong DataFrame thứ nhất và thêm các cột từ DataFrame 2 bên cạnh cột khớp với `on`
* **Right**: Ngược lại, giữ tất cả hàng trong DataFrame thứ hai và thêm các cột từ DataFrame thứ nhất. 
* **Inner**: Chỉ giữ những hàng trong DataFrame thứ nhất giao với DataFrame 2  
* **Outer**: Giữ tất cả hàng trong cả 2 DataFrame bất kể có giá trị khớp với nhau hay không.

![Ảnh](https://miro.medium.com/max/669/1*DhTppnBct_rr9QMX_cHXNg.png)

In [ ]:
stocks = pd.read_csv('https://gist.githubusercontent.com/alexdebrie/b3f40efc3dd7664df5a20f5eee85e854/raw/ee3e6feccba2464cbbc2e185fb17961c53d2a7f5/stocks.csv')
stocks.info()

In [ ]:
stocks2 = pd.read_csv('https://gist.githubusercontent.com/alexdebrie/53ebac036b404875ef8e981c0cbd0901/raw/6c70336130eb7e45cec167ee7cd52d15baa392ea/stocks2.csv')
combined = pd.concat([stocks, stocks2], ignore_index=True)
combined

In [ ]:
companies = pd.read_csv('https://gist.githubusercontent.com/alexdebrie/90954a10bbdebd89185f7b4d340563cd/raw/d91744e599356b7a52b9c027751d4613c06c5f40/companies.csv')
companies

In [ ]:
enriched = combined.join(companies.set_index('symbol'), on='symbol', how='left')
enriched

Ta được kết quả có 25 hàng với 3 cột mới chỉ áp dụng với 3 cty AMZN, AAPL, and GOOG. Bây giờ so sánh với kết quả khi dùng inner join

In [ ]:
enriched_inner = combined.join(companies.set_index('symbol'), on='symbol', how='inner')
enriched_inner

Trong trường hợp tên cột của 2 datasets trùng nhau, ta sẽ phải dùng thêm tham số `lsuffix` và `rsuffix` để phân biệt chúng.
## Bài tập thực hành
1. Đổi tên 2 cột `region_1` và `region_2` thành `region` và `locale`
2. Đặt tên index của dataset là `wines`
3. [Powerlifting dataset](https://drive.google.com/open?id=1hmwQ4DPv7w0U8AjkoAEzbMc3FR6jGGKp) chứa một bảng CSV có các cuộc thi đấu tạ và một bảng khác chứa danh sách các vận động viên. Cả 2 bảng đều chứa cột `MeetID` một khóa độc nhất cho mỗi cuộc thi trong dataset. Hãy tạo một dataset gộp 2 bảng này vào thành một.

In [ ]:
powerlifting_meets = pd.read_csv("../input/powerlifting-database/meets.csv")
powerlifting_competitors = pd.read_csv("../input/powerlifting-database/openpowerlifting.csv")
powerlifting_meets.info()
powerlifting_competitors.info()